# Multiple Form 13 Filings Proof of Concept

The goal of this is to write a program that downloads the form 13 filings from hedge funds and exports them in a csv file. This csv file can then be used to drive other analysis. The csv should contain the last 4 years of filings, this means there will be 12 dates present. 

In [291]:
import pandas as pd
import numpy as np

In [292]:

import urllib.request
import requests

In [293]:
from bs4 import BeautifulSoup

In [294]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

The code can be pulled from the sec website using the bs4 scraping program that is already written. The various periods and url links can be stored in a dict where they can be referenced by the program. 

## Notes
> It is important to note that there is a more elegant solution to the problem of matching the raw output from the SEC edgar website to the associated ticker. In my opinion, this is the use of CUSIP numbers, these are listed on the SEC filings. The problem however is that CUSIP numbers are not publically available. The ideal system would be a .csv with CUSIP numbers, associated company name, and stock symbol. This would allow fast and highly accurate matching with little duplicate datapoints. I was not able to use this approach because CUSIP data is expensive. One problem with my approach is the required level of matching accuracy. (0.70 for the function below) This number determines whether a match is included in the result based on the percent match to the exchange company names. Through trial and error, I have found that 0.70 works well because it eliminates most of the incorrect matches but allows the code to approximate most of the correct matches as well. In the example below, there is a duplicate values in ABEO, this is due to the NLP matching. If the accuracy benchmark (currently set at 0.70) is decreased to 0.60, it excludes datapoints that were correctly generated without removing the duplicate value of ABEO. This basically means that one company name is close enough to that of another name that they are matched incorrectly with a higher degree of accuracy than some names that are matched correctly. This is a fundamental risk to the use of NLP and TDIFD for name matching. 

### *The code below is the modified ticker matching function, the modifications include processing the tickers to remove duplicates, sum those values, and remove companies that are associated with puts.* 

In [296]:
""" These .csv files were downloaded from the NASDAQ and NYSE websites and are public """

# this is the creation of the df that the functions will reference
nasdaq = pd.read_csv('NASDAQ.csv') #imports the NASDAQ tickers
nyse = pd.read_csv('NYSE.csv') #imports the NYSE tickers
combEx = nasdaq.append(nyse,ignore_index=True) #this is the combination of the data from the nasdaq and nyse

In [297]:
# this is the version of the pull urls from sec code that will be used in this program. important to note that since book values are being included, ticker nlp matches that are below the accuracy level need to be replaced with error or another identifier so that the corresponding book value pct can also be removed using list comp. 

def pullSec2(url):
    """ pulls both the name of the company and the book value from the sec """
    names = []
    book_value = []
    source = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(source, 'xml')
    FormData = soup.find_all('td', class_="FormData")
    for xx in FormData[0:]:
        res = xx.text.strip()
        names.append(res) # creates list of raw outputs, un-uniform outputs, contains other info

    FormDataR = soup.find_all('td', class_="FormDataR")
    for aa in FormDataR[0:]:
        res1 = aa.text.strip()
        book_value.append(res1)
    book_value = book_value[0::5] # list of strings that contain the values of the positions 

    """ DATA PROCESSING """

    """ Creating list of Company names """ # if a Put, then Put is added to the list after the name of the company. 
    cleanNames = [a for a in names if a != 'Put' and a != 'Call'] # removes options from the raw output, this is to create uniform ordering
    cleanNames = cleanNames[0::5] # selects the company names from the list based on location in list
    namesDeriv = [a for a in names if a in cleanNames or a == 'Put'] # pulls the company names and options out of the list in the correct order, if name associated with Put then Put added to list after the name of the company 
    """ Replacing names associated with Put options with 9999 """ # this is done to preserve the len of the list and keep it equal to the len of the list of values 
    names = []
    for i in range(0,len(namesDeriv)-1): # iterate through index of the list
        if namesDeriv[i+1] == 'Put': # check if next item in list is put or if current item in list is, 
            names.append(9999) # if the next item in the list is Put, then append 9999 to the list, this takes the place of the company name associated with the Put 
        elif namesDeriv[i+1] != 'Put' and namesDeriv[i] != 'Put': # if company does not have put after it then add to list, not allow str "puts" to be appended 
            names.append(namesDeriv[i])

    """ Note the method above does not count the last item in the list, the code below fixes this """
    if namesDeriv[len(namesDeriv)-1] != 'Put': # checks if last item in the list is a Put
        names.append(namesDeriv[len(namesDeriv)-1]) # if last item in the list is not a Put, then add last item to the list

    """ Creation of Pct Book values and forming tuples """
    intValues = [int(a.replace(',','')) for a in values] # removes commas from strings in list, converts to int 
    bookPct = [round(a/sum(intValues),3) for a in intValues] # creates new list of each value as a percentage of the total
    tuples = list(zip(names,bookPct)) # this is a list of tuples that contain the data 
    tuples = [(a,b) for a,b in tuples if a != 9999] # this removes the names and values that had puts and were replaced with 9999, tuple was created so that book the associated book value could be removed with the 9999 list entry. 

    """ There are duplicate values in the dataset, this can be fixed using dict """
    result = {} # sets up dict 
    for name,value in tuples: # iterate through list
        total = result.get(name,0) + value # find sum of duplicate values
        result[name] = total

    """ Final output -- Names and Values of equal len """
    # below lists should be of equal len
    cleanNames = [a for a in result.keys()] # names of the companies  
    cleanValues = [a for a in result.values()] # pct book values 

    """ This part matches the raw names from SEC to a list of all NYSE and NASDAQ companies and tickers using NLP """
    # this re stuff is needed for the NLP below. note, this does not have to be in function, can be somewhere else, just here for ease and clarity. 
    def ngrams(string, n=3):
        string = string.encode("ascii", errors="ignore").decode() 
        string = string.lower()
        chars_to_remove = [')', '(', '.', '|', '[', ']', '{', '}', "'"]
        rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
        string = re.sub(rx, '', string) # remove the list of chars defined above
        string = string.replace('&', 'and')
        string = string.replace(',', ' ').replace('-', ' ')
        string = string.title() # Capital at start of each word
        string = re.sub(' +',' ',string).strip() # combine whitespace
        string = ' ' + string + ' ' # pad
        string = re.sub(r'[,-./]|\sBD', r'', string)
        ngrams = zip(*[string[i:] for i in range(n)])
        return [''.join(ngram) for ngram in ngrams]

    # pulls data from the NYSE/NASDAQ list for names of companies. 
    exchangeNames = combEx['Name'].to_list() # this is list2, list1 is the raw names from SEC, important to note that this must be interchanged, can be imported from csv, etc. 

    # this is the NLP stuff that finds the matches. 
    """For each item in list1, find the match in list2"""
    list1 = cleanNames # these are so the code can be adjusted easier for other list inputs. List 1/2 are referenced in the code below. names from the input 
    list2 = exchangeNames # names to matched against, from exchanges 

    vectorizer = TfidfVectorizer(analyzer=ngrams, lowercase=False)
    tfidf = vectorizer.fit_transform(list2)
    nbrs = NearestNeighbors(n_neighbors=1, n_jobs=-1).fit(tfidf)
    distances, indices = nbrs.kneighbors(vectorizer.transform(list1))
    
    matches = [(round(distances[i][0], 2), list1[i], list2[j[0]]) for i, j in enumerate(indices)]
    matches = pd.DataFrame(matches, 
                           columns=['score', 'original', 'matched'])
    
    """ This code finds the matching ticker to the 'matched' cleaned name from the exchange dataframe. """
    matches['bookPct'] = cleanValues
    matches = matches.loc[matches['score'] <= 0.70]
    resDf = pd.DataFrame({'matches':matches['matched'].to_list(),'bookPct':matches['bookPct'].to_list()})
    refTickers = combEx[['Name','Symbol']]
    resDf = resDf.join(refTickers.set_index('Name'),on='matches')
    resDf = resDf.drop(['matches'],axis=1)
    resDf = resDf.reindex(columns=['Symbol','bookPct'])
    return resDf

In [298]:
test = pullSec2(testUrl) # this is just a test of the functions ability to pull names, and bookPct
print(test.shape)
test.head()

(124, 2)


,Symbol,bookPct
0,ONEM,0.001
1,ACCD,0.002
2,ADAP,0.000
3,ADPT,0.006
4,ADMA,0.006


In [299]:
len(test['Symbol'].to_list()) == len(test['Symbol'].unique().tolist()) # this means that there are no duplicate values in the dataframe. -- the update to the function was successful

True

## The above code works

The above function works, it removes duplicates, names associated with puts, and calculates the bookPct of each positions before these eliminations. This means that the book Pct really represents the actual conviction of the fund and not a subset. This dataset represents only the tickers that they are focusing on for long exposure. 

The code below are urls and periods. This list will be iterated through. 

In [300]:
# this is the list that contains the string with the date of the event and the url to the xml edgar page
# note that this particular proof of concept is for Perc Adv -- this is because their book is the most complex, other ones should be easy compared to this one. 

""" This is the program that creates the list the stores the periods and urls that will be scraped. """
# note the text to the right of each line is the period and the date of filing. -- once again all these urls are for perc adv

urls = [
    'https://www.sec.gov/Archives/edgar/data/1224962/000101297520000835/xslForm13F_X01/infotable.xml', #q320 - 2020-11-16
    'https://www.sec.gov/Archives/edgar/data/1224962/000101297520000701/xslForm13F_X01/infotable.xml', #q220 - 2020-08-17
    'https://www.sec.gov/Archives/edgar/data/1224962/000101297520000472/xslForm13F_X01/infotable.xml', #q120 - 2020-05-18
    'https://www.sec.gov/Archives/edgar/data/1224962/000101297520000250/xslForm13F_X01/perceptive.xml', #q419 - 2020-02-19
    'https://www.sec.gov/Archives/edgar/data/1224962/000101297519000707/xslForm13F_X01/infotable.xml', #q319 - 2019-11-14
    'https://www.sec.gov/Archives/edgar/data/1224962/000101297519000583/xslForm13F_X01/infotable.xml', #q219 - 2019-08-14
    'https://www.sec.gov/Archives/edgar/data/1224962/000101297519000380/xslForm13F_X01/infotable.xml', #q119 - 2019-05-16
    'https://www.sec.gov/Archives/edgar/data/1224962/000101297519000177/xslForm13F_X01/infotable.xml', #q418 - 2019-02-14
    'https://www.sec.gov/Archives/edgar/data/1224962/000101297518001239/xslForm13F_X01/infotable.xml', #q318 - 2018-11-14
    'https://www.sec.gov/Archives/edgar/data/1224962/000101297518001030/xslForm13F_X01/infotable.xml', #q218 - 2018-08-14
    'https://www.sec.gov/Archives/edgar/data/1224962/000101297519000380/xslForm13F_X01/infotable.xml', #q118 - 2018-05-15
    'https://www.sec.gov/Archives/edgar/data/1224962/000101297518000171/xslForm13F_X01/infotable.xml' #q417 - 2018-02-14
    ] 

periods = [ # this is the list of periods that are covered in the analysis. -- this is the same for every fund. 
    'Q320',
    'Q220',
    'Q120',
    'Q419',
    'Q319',
    'Q219',
    'Q119',
    'Q418',
    'Q318',
    'Q218',
    'Q118',
    'Q417'
    ]

In [301]:
# the goal of this cell is to create a way to iterate over an unknown number 

""" Note that this requires the previous function pullSec """
def createCsv2(periods, urls):
    res = [] # this is just an empty list that the dataframes can be stored in
    for a in urls: # this iteration pulls a total list of all the tickers over that period of time, book value is dropped. 
        data = pullSec2(a) # this references the function that is defined earlier. 
        res.append(data)

    tickers = []
    for a in res: # this references each res df that is present in the master list 
        ticker = a['Symbol'].to_list()
        tickers.append(ticker)

    flatTickers = [item for sublist in tickers for item in sublist] # flattens the list of tickers 
    finalTickers = np.unique(np.array(flatTickers)).tolist() # removes the duplicates from the list
    masterDf = pd.DataFrame({'Symbol':finalTickers}) # this is the master dataframe that each fq will be matched against with the book values -- unique values
    for a in [a for a in range(len(periods))]: # this renames each bookPct col to be the corresponding period
        res[a].columns = ['Symbol',periods[a]]
    res.insert(0,masterDf) # this just inserts the dataframe into the 0 position.
    dfs = [df.set_index('Symbol') for df in res]
    output = dfs[0].join(dfs[1:])
    return output

In [302]:
output = createCsv2(periods,urls)

In [303]:
output.head()

,Q320,Q220,Q120,Q419,Q319,Q219,Q119,Q418,Q318,Q218,Q118,Q417
Symbol,,,,,,,,,,,,
ABEO,NaN,NaN,0.0,0.002,0.002,0.001,0.003,0.003,0.003,0.001,0.003,0.003
ABEO,NaN,NaN,0.0,0.002,0.002,0.001,0.003,0.003,0.003,0.002,0.003,0.003
ABMD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,NaN,0.000
ACAD,NaN,NaN,NaN,0.000,NaN,NaN,0.000,NaN,NaN,NaN,0.000,NaN
ACCD,0.002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [304]:
output.shape # 312, 12 @ 0.70-- this is the shape of the df before the accuracy was increased (309,12 @ 0.65 ) 
""" This shows that increasing the accuracy matching benchmark actually removes non-duplicates from the dataset but is not able to remove the ABEO duplicate """

' This shows that increasing the accuracy matching benchmark actually removes non-duplicates from the dataset but is not able to remove the ABEO duplicate '

When the matching percent was decreased to 0.65, (makes the barrier for inclusion higher), the shape of the df is decreased by 3 rows, note that before the only duplicate datapoints are the ABEO rows, this means that lowering this benchmark had decreased the number of unique inclusions in the result. This is a problem. Further, the ABEO duplicate datapoint is still present. 

In [305]:
output

,Q320,Q220,Q120,Q419,Q319,Q219,Q119,Q418,Q318,Q218,Q118,Q417
Symbol,,,,,,,,,,,,
ABEO,NaN,NaN,0.000,0.002,0.002,0.001,0.003,0.003,0.003,0.001,0.003,0.003
ABEO,NaN,NaN,0.000,0.002,0.002,0.001,0.003,0.003,0.003,0.002,0.003,0.003
ABMD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,NaN,0.000
ACAD,NaN,NaN,NaN,0.000,NaN,NaN,0.000,NaN,NaN,NaN,0.000,NaN
ACCD,0.002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
ZBH,NaN,NaN,0.001,0.000,0.002,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZGNX,NaN,0.003,0.000,0.004,0.001,0.003,0.024,0.002,0.002,0.030,0.024,NaN
ZIOP,NaN,NaN,NaN,NaN,NaN,0.002,NaN,NaN,NaN,NaN,NaN,NaN


In [306]:
# this is the sum of pct book each q 
theSum = output.sum(axis=0)

In [307]:
theSum

Q320    0.996
Q220    0.903
Q120    0.858
Q419    0.900
Q319    0.791
Q219    0.846
Q119    0.802
Q418    0.803
Q318    0.701
Q218    0.767
Q118    0.802
Q417    0.748
dtype: float64

In [308]:
mean = output.mean(axis=0).round(decimals=3) # this is the mean pct book allocated to each position for each quarter 
mean

Q320    0.008
Q220    0.007
Q120    0.008
Q419    0.008
Q319    0.008
Q219    0.009
Q119    0.008
Q418    0.009
Q318    0.007
Q218    0.008
Q118    0.008
Q417    0.008
dtype: float64

In [309]:
noNan = output[~output.isnull().any(axis=1)] # this is a df of only rows with no nan values

In [310]:
print(noNan.shape) # this is a very small subset of the data, indicates the perc rarely increases q/q
noNan # this is the rows of the dataframe where there are no nan values

(18, 12)


,Q320,Q220,Q120,Q419,Q319,Q219,Q119,Q418,Q318,Q218,Q118,Q417
Symbol,,,,,,,,,,,,
ADMA,0.006,0.006,0.007,0.007,0.006,0.006,0.006,0.006,0.011,0.007,0.006,0.007
ALBO,0.011,0.011,0.007,0.007,0.000,0.000,0.007,0.011,0.006,0.001,0.007,0.052
ALDX,0.007,0.007,0.006,0.006,0.011,0.011,0.006,0.007,0.001,0.001,0.006,0.001
ARNA,0.007,0.024,0.008,0.001,0.024,0.007,0.008,0.001,0.001,0.000,0.008,0.009
BPMC,0.009,0.001,0.001,0.010,0.037,0.037,0.024,0.037,0.010,0.032,0.024,0.016
CTMX,0.003,0.003,0.008,0.032,0.008,0.003,0.003,0.003,0.003,0.009,0.003,0.003
EIGR,0.000,0.002,0.000,0.005,0.000,0.002,0.005,0.002,0.005,0.003,0.005,0.001
FOLD,0.052,0.052,0.001,0.001,0.001,0.052,0.001,0.007,0.007,0.005,0.001,0.005
GBT,0.047,0.002,0.009,0.000,0.009,0.002,0.003,0.002,0.002,0.003,0.003,0.009


The small percentage of the total companies that have no NaN values in any FQ indicate that there are few companies that Perc Adv have strong Q/Q increases in over a long period. 

In [311]:
len(output.index) == len(output.index.unique()) # means that there is a duplicate in the dataset 

False

In [312]:
len(output.index) - len(output.index.unique()) # this is the difference between the len of the index and unique values, if the len of the lists are equal then there are no duplicates, otherwise there are duplicates, this shows how many duplicates there are

1

In [313]:
""" simple function to find duplicates in the dataset """
unique = [] 
duplicates = []
for a in output.index.tolist():
    if a not in unique:
        unique.append(a)
    else:
        duplicates.append(a)

In [314]:
duplicates # this is the duplicate value

['ABEO']

In [320]:
output.to_csv('PercAdvQ-Q113s.csv') # saves the final output as .csv for further analysis 

## Conclusions
This code represents a proof of concept for pulling multiple quarters of SEC filings and compiling them into a single dataset for further analysis. This is an imperfect method that would be much improved with access to CUSIP datasets. There are some duplicates in the final dataset likely due to the NLP TDIFD matching but this is a problem I will continue to work. The project overall is a success as this simple datastructure can be exported to .csv and then imported into other code for further analysis. The hardcoded urls and periods used in this code could likely be automated using a web crawler. That is really out of the scope of this project as this is just proof of concept. However, it warrants further research. 